# Calculating Growth, DCF models

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#pull the income statement, balance sheet, and cash flow statement as downloaded from stockrow.com
ticker = 'AAPL'
ticker_inc_df = pd.read_excel(f'./Income/{ticker}_inc.xlsx').T
ticker_bal_df = pd.read_excel(f'./BalanceSheet/{ticker}_bal.xlsx').T
ticker_cfl_df = pd.read_excel(f'./CashFlow/{ticker}_cashflow.xlsx').T

# change the first row to be the column headers
ticker_inc_df.columns = ticker_inc_df.iloc[0]
ticker_bal_df.columns = ticker_bal_df.iloc[0]
ticker_cfl_df.columns = ticker_cfl_df.iloc[0]

#remove the first row
ticker_inc_df = ticker_inc_df.iloc[1:]
ticker_bal_df = ticker_bal_df.iloc[1:]
ticker_cfl_df = ticker_cfl_df.iloc[1:]

ticker_inc_df.head(2)

#### Some data cleaning
Set columns we want, combine into one DF, create a field for FreeCashFlow (FCF)

In [ ]:
# these are the columns we're interested in
inc_columns = ['Revenue','Gross Profit','Operating Income','Income Tax Provision', 'Net Income Common']
bal_columns = ['Total Assets','Total current assets','Total liabilities','Total current liabilities',
               'Shareholders Equity (Total)']#,'Shares (Common)']
cfl_columns = ['Dividends Paid (Common)','Capital expenditures','Operating Cash Flow']#,'Equity Repurchase (Common, Net)']

# this will apply only the columns above to each of our financial statement DataFrames
ticker_inc_df = ticker_inc_df[inc_columns]
ticker_bal_df = ticker_bal_df[bal_columns]
ticker_cfl_df = ticker_cfl_df[cfl_columns]

# this will combine or concatenate our three DataFrames into one
ticker_df = pd.concat([ticker_inc_df,ticker_bal_df,ticker_cfl_df], axis=1)

# defining a new column for FCF
ticker_df['FreeCashFlow'] = ticker_df['Operating Cash Flow'] - ticker_df['Capital expenditures']

ticker_df.head(2)

In [ ]:
# TTM - Revenue
plt.figure(figsize=(9,6))
sns.scatterplot(x=ticker_df.index,y=ticker_df['Revenue'])
plt.title('Trailing 12 months of Revenue for company')
plt.ylabel('Revenue ($100B)');

## Growth rates 1
This method determines an average Revenue growth rate by taking the average of all 1-yr Revenue growth rates at t = 0, t = -1, t = -2 etc

In [ ]:
# averaging all cumulative 1-yr revenue growths
rev_growth_list = [((ticker_df['Revenue'][i] - ticker_df['Revenue'][i+4]) / ticker_df['Revenue'][i+4]) 
                   for i in range(0,len(ticker_df['Revenue'])-4)]

print(f"Avg 1-yr Revenue growth rate: {round(sum(rev_growth_list)/len(rev_growth_list)*100,2)}%")

##### Same for Free Cash Flow:

In [ ]:
# averaging all FCF growth rates
fcf_growth_list1 = [((ticker_df['FreeCashFlow'][i] - ticker_df['FreeCashFlow'][i+4]) / ticker_df['FreeCashFlow'][i+4]) 
                   for i in range(0,len(ticker_df['FreeCashFlow'])-4)]

fcf_growth_rate1 = sum(fcf_growth_list1)/len(fcf_growth_list1)
print(f"Free Cash Flow average 1-yr growth rate: {round(fcf_growth_rate1*100,2)}%")

##### Same for *every* financial metric

In [ ]:
#calculating all cumulative 1-yr average growths at once
growth_list1 = []
for col in ticker_df.columns:
    try:
        col_growth_list1 = [((ticker_df[col][i] - ticker_df[col][i+4]) / ticker_df[col][i+4]) 
                           for i in range(0,len(ticker_df[col])-4)]
        print(f"Avg growth for {col} is {sum(col_growth_list1)/len(col_growth_list1)}")
        growth_list1.append(sum(col_growth_list1)/len(col_growth_list1))
    except:
        print(f"{col} throws an error")

all_metrics_growth_rate1 = sum(growth_list1)/len(growth_list1) 
print(f"Avg growth: {all_metrics_growth_rate1}")

In [ ]:
# this code sorts the growth rates from low to high, 
# then it removes the top 2 and lowest 2
growth_list1.sort()
normalized_ticker_growth1 = sum(growth_list1[2:-2]) / len(growth_list1[2:-2])

print(f"The 'normalized' growth rate for (almost all) metrics: {round(normalized_ticker_growth1*100,2)}%")

## Growth rates 2
This method determines an average Revenue growth rate by taking the 1-yr, 2-yr, 3-yr etc. growth rates and averaging them all together

In [ ]:
#this version only considers growth rate starting from t = 0, looking backwards at quarterly data
fcf_growth_list2 = [((1+((ticker_df['FreeCashFlow'][0] - ticker_df['FreeCashFlow'][i+4]) / ticker_df['FreeCashFlow'][i+4]))**(1/(1+i/4))-1) 
                    for i in range(len(ticker_df['FreeCashFlow'])-4)]

fcf_growth_rate2 = sum(fcf_growth_list2)/len(fcf_growth_list2)
print(f"Free Cash Flow avg growth rate last decade: {round(fcf_growth_rate2*100,2)}%")

In [ ]:
growth_list2 = []
for col in ticker_df.columns:
    try:
        col_growth_list2 = [((1+((ticker_df[col][0] - ticker_df[col][i+4]) / ticker_df[col][i+4]))**(1/(1+i/4))-1) 
                            for i in range(len(ticker_df[col])-4)]
        print(f"Avg growth for {col} is {sum(col_growth_list2)/len(col_growth_list2)}")
        growth_list2.append(sum(col_growth_list2)/len(col_growth_list2))
    except:
        print(f'{col} throws an error')
        
all_metrics_growth_rate2 = sum(growth_list2)/len(growth_list2) 
print(f"Avg growth of all metrics: {all_metrics_growth_rate2}")

In [ ]:
# this code sorts the growth rates from low to high, 
# then it removes the top 2 and lowest 2
growth_list2.sort()
normalized_ticker_growth2 = sum(growth_list2[2:-2]) / len(growth_list2[2:-2])

print(f"The 'normalized' growth rate for (almost all) metrics: {round(normalized_ticker_growth2*100,2)}%")

## DCF Models

In [ ]:
# DCF using the treasury rate as the discount rate and avg. FCF growth as growth rate
discount_rate = 0.0163

def dcf_maker(ticker_df,growth_rate,discount_rate,years):
    # we start at zero, then incrementally add each subsequent year's FCF
    fcf_over_time = 0
    # our base will be the most recent year's FCF
    fcf_start = ticker_df['FreeCashFlow'][0]

    # covering a range of 10 years
    for i in range(1,years+1):
        fcf_over_time += fcf_start * (1+growth_rate)**i / (1+discount_rate)**i

    print(f"Total FCF: ${round(fcf_over_time/1_000_000_000,1)} billion")
    return fcf_over_time

In [ ]:
test1 = dcf_maker(ticker_df=ticker_df,growth_rate=0.1,discount_rate=0.0163,years=10)

In [ ]:
all_growth_rates = [fcf_growth_rate1,fcf_growth_rate2,all_metrics_growth_rate1,all_metrics_growth_rate2,
                    normalized_ticker_growth1, normalized_ticker_growth2]
discount_list = np.linspace(0.01,0.08,20).tolist()
year_list = [8,9,10,11,12]

fcf_values_list = []
for rate in all_growth_rates:
    for discount in discount_list:
        for year in year_list:
            #print(f"Growth rate: {round(rate*100,2)}%, Discount rate: {round(discount*100,2)}%, Years: {year}")
            fcf_values_list.append(dcf_maker(ticker_df,rate,discount,year))

In [ ]:
sum(fcf_values_list) / len(fcf_values_list) / 1000000000

In [ ]:
fcf_values_list.sort
len(fcf_values_list)
plt.hist(fcf_values_list,bins=50);

In [ ]:
from bokeh.io import show, output_file
from bokeh.plotting import figure

data = fcf_values_list
hist, edges = np.histogram(data, density=True, bins=50)

p = figure()
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")

output_file("hist.html")
show(p)

## More Random / Ancillary Calculations

In [ ]:
growth_rate = fcf_growth_rate1
discount_rate = 0.05
years=15

dcf_maker(ticker_df,growth_rate,discount_rate,years)

In [ ]:
np.linspace(0.01,0.15,15)

In [ ]:
# Calculate most recent FCF, next year FCF, and discount next year's FCF
# discount_rate is treasury rate here
bill = 1_000_000_000
discount_rate = 0.0163

print(f"Most recent FCF: ${round(ticker_df['FreeCashFlow'][0]/bill,2)} billion")
print(f"Next year FCF: ${round(ticker_df['FreeCashFlow'][0]*(1+fcf_growth_rate1)/bill,2)} billion")
print(f"Next year FCF, discounted: ${round(ticker_df['FreeCashFlow'][0]*(1+fcf_growth_rate1)**(1)/(1+discount_rate)/bill,2)} billion")

In [ ]:
# DCF using the treasury rate as the discount rate and avg. FCF growth as growth rate
treasury_rate = 0.0163
# we start at zero, then incrementally add each subsequent year's FCF
fcf_over_time1 = 0
# our base will be the most recent year's FCF
fcf_start = ticker_df['FreeCashFlow'][0]

# covering a range of 10 years
for i in range(1,10+1):
    fcf_over_time1 += fcf_start * (1+fcf_growth_rate1)**i / (1+treasury_rate)**i
    print(f"Year {i}: ${round(fcf_start * (1+fcf_growth_rate1)**i / (1+treasury_rate)**i / 1_000_000_000,1)} billion")

print(f"Total FCF: ${round(fcf_over_time1/1_000_000_000,1)} billion")

In [ ]:
# DCF using 5% as the discount rate and avg. FCF growth as growth rate
discount_rate = 0.05
# we start at zero, then incrementally add each subsequent year's FCF
fcf_over_time = 0
# our base will be the most recent year's FCF
fcf_start = ticker_df['FreeCashFlow'][0]

# covering a range of 15 years
for i in range(1,15+1):
    fcf_over_time += fcf_start * (1+fcf_growth_rate1)**i / (1+discount_rate)**i
    print(f"Year {i}: ${round(fcf_start * (1+fcf_growth_rate1)**i / (1+discount_rate)**i / 1_000_000_000,1)} billion")

print(f"Total FCF: ${round(fcf_over_time/1_000_000_000,1)} billion")

In [ ]:
# turning FCF into per-share stock price predictions
ticker_fcf1 = round(fcf_over_time1/ticker_df['Shares (Common)'][0],2)
ticker_fcf2 = round(fcf_over_time/ticker_df['Shares (Common)'][0],2)

avg_fcf = round((ticker_fcf1 + ticker_fcf2)/2,2)

print(f"Total FCF per share, v1: ${ticker_fcf1}")
print(f"Total FCF per share, v2: ${ticker_fcf2}")
print(f"Average FCF per share: ${avg_fcf}")

In [ ]:
# DCF using treasury as discount rate and all AAPL growth as growth rate
treasury_rate = 0.05
fcf_over_time = 0
fcf_start = ticker_df['FreeCashFlow'][0]
for i in range(1,15+1):
    fcf_over_time += fcf_start * (1+normalized_ticker_growth)**i / (1+treasury_rate)**i

fcf_over_time/1_000_000_000

In [ ]:
# if total growth over 5 years is 180%
# then the growth rate is:
growth_180 = 1.8**(1/5)
growth_180

In [ ]:
for i in range(0,25):
    print(f"{1+i/4} - {1.80**(1/(1+i/4))}")